# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
file_path = '../WeatherPy/output_data/cities.csv'
city_df = pd.read_csv(file_path)
city_df.head()

,Unnamed: 0,City Name,Latitude,Longitude,Temperature,Humidity,Wind Speed,Cloudiness
0,0,Tuatapere,-46.13,167.68,288.15,92,6.71,100
1,1,Ushuaia,-54.80,-68.30,278.15,93,10.80,75
2,2,Ilulissat,69.22,-51.10,270.15,49,7.70,81
3,3,Barrow,71.29,-156.79,251.15,69,10.80,1
4,4,Saskylakh,71.92,114.08,237.37,83,1.62,23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#Configure gmaps with API Key
gmaps.configure(api_key=g_key)

In [10]:
#Store Latitude and Longitude into Locations
locations = city_df[["Latitude", "Longitude"]].astype(float)
humid = city_df["Humidity"]

In [11]:
#Add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
